In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import densenet121, DenseNet121_Weights
from torchvision.models import densenet169, DenseNet169_Weights
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader
import torch.optim.lr_scheduler as lr_scheduler

print("Testing DenseNet121 implementation")

Testing DenseNet121 implementation


In [3]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train = datasets.ImageFolder(root='MAIN_DS/10B-3D-Uni/train', transform=transform)
val = datasets.ImageFolder(root='MAIN_DS/10B-3D-Uni/val', transform=transform)
test = datasets.ImageFolder(root='MAIN_DS/10B-3D-Uni/test', transform=transform)

train_loader = DataLoader(train, batch_size=16, shuffle=True)
val_loader = DataLoader(val, batch_size=16, shuffle=True)
test_loader = DataLoader(test, batch_size=16, shuffle=False)

print(f"Training set size: {len(train)} images")
print(f"Validation set size: {len(val)} images")
print(f"Testing set size: {len(test)} images")


Training set size: 1200 images
Validation set size: 400 images
Testing set size: 401 images


In [5]:

densenet = densenet121(weights=DenseNet121_Weights.IMAGENET1K_V1)

num_features = densenet.classifier.in_features
densenet.classifier = nn.Linear(num_features, 2)  # Binary classification

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
densenet = densenet.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(densenet.parameters(), lr=0.001, weight_decay=0.01)

Using device: cuda


In [7]:
def evaluate_model(model, data_loader):
    """Evaluate model on the provided data loader"""
    model.eval()
    correct = 0
    total = 0
    running_loss = 0.0
    
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = correct / total * 100
    avg_loss = running_loss / len(data_loader)
    return accuracy, avg_loss

In [9]:

num_epochs = 30
best_val_loss = float('inf')
patience = 8 
patience_counter = 0 
best_model_weights = None

scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=3)

print("Starting training with DenseNet121...")

for epoch in range(num_epochs):
    # Training phase
    densenet.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = densenet(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    train_accuracy = correct / total * 100
    train_loss = running_loss / len(train_loader)
    
    # Validation phase
    val_accuracy, val_loss = evaluate_model(densenet, val_loader)
    
    # Save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0  # Reset the patience counter
        best_model_weights = densenet.state_dict()
    else:
        patience_counter += 1  # Increment the patience counter
        print(f"Patience counter {patience_counter}")
    
    # Step the scheduler with validation loss
    scheduler.step(val_loss)
    
    print(f'Epoch [{epoch + 1}/{num_epochs}]')
    print(f'Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.2f}%')
    print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%')
    print(f'Current LR: {optimizer.param_groups[0]["lr"]:.6f}')
    print('-' * 60)

    if patience_counter >= patience:
        print(f"Early stopping at epoch {epoch + 1}. Best validation loss: {best_val_loss:.4f}")
        break

torch.save(best_model_weights, 'saved_models/Densenet121/10B-3D-Uni2_densenet121.pth')
print("Training completed.")

Starting training with DenseNet121...
Epoch [1/30]
Training Loss: 0.6979, Training Accuracy: 58.25%
Validation Loss: 1.0318, Validation Accuracy: 54.00%
Current LR: 0.001000
------------------------------------------------------------
Epoch [2/30]
Training Loss: 0.6611, Training Accuracy: 60.83%
Validation Loss: 0.8573, Validation Accuracy: 60.50%
Current LR: 0.001000
------------------------------------------------------------
Epoch [3/30]
Training Loss: 0.6507, Training Accuracy: 62.25%
Validation Loss: 0.6190, Validation Accuracy: 63.00%
Current LR: 0.001000
------------------------------------------------------------
Patience counter 1
Epoch [4/30]
Training Loss: 0.6553, Training Accuracy: 61.58%
Validation Loss: 0.6249, Validation Accuracy: 64.00%
Current LR: 0.001000
------------------------------------------------------------
Epoch [5/30]
Training Loss: 0.6098, Training Accuracy: 65.92%
Validation Loss: 0.5614, Validation Accuracy: 70.25%
Current LR: 0.001000
-------------------

In [11]:
model = densenet121(weights=None)
model.classifier = nn.Linear(model.classifier.in_features, 2)

model.load_state_dict(torch.load('saved_models/Densenet121/10B-3D-Uni2_densenet121.pth'))

model = model.to(device)


test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test = datasets.ImageFolder(root='MAIN_DS/10B-3D-Uni/test', transform=test_transform

test_loader = DataLoader(test, batch_size=16, shuffle=False)

test_accuracy, test_loss = evaluate_model(model, test_loader)
print(f'\nFinal Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')

C:\Users\ajeet\AppData\Local\Temp\ipykernel_19304\3254670.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('saved_models/Densenet121/10B-


Final Test Loss: 0.2686, Test Accuracy: 88.78%
